In [6]:
import caveclient
import pandas as pd
import numpy as np
import json
from caveclient import CAVEclient



In [31]:
# I want the user to be able to insert their own dataset from the drop down menu outputs from the list in: client.info.get_datastacks()

client = caveclient.CAVEclient('minnie65_phase3_v1') 
ct_df=client.materialize.query_table('aibs_metamodel_mtypes_v661')
ct_df
# spec of a segment info file here
# https://github.com/google/neuroglancer/blob/master/src/datasource/precomputed/segment_properties.md



,id_ref,created_ref,valid_ref,volume,pt_supervoxel_id,pt_root_id,id,created,valid,target_id,classification_system,cell_type,pt_position,bb_start_position,bb_end_position
0,102922,2020-09-28 22:41:02.615344+00:00,t,238.492877,78747024056573217,864691135367507449,1,2023-08-17 06:30:26.148393+00:00,t,102922,excitatory_neuron,L5a,"[101552, 93584, 24703]","[nan, nan, nan]","[nan, nan, nan]"
1,103918,2020-09-28 22:43:15.936278+00:00,t,248.065065,78256572010460990,864691135697301914,2,2023-08-17 06:30:26.149174+00:00,t,103918,inhibitory_neuron,STC,"[97968, 109616, 16139]","[nan, nan, nan]","[nan, nan, nan]"
2,104138,2020-09-28 22:44:51.435738+00:00,t,289.790034,78678097280564305,864691136143975220,3,2023-08-17 06:30:26.149917+00:00,t,104138,inhibitory_neuron,STC,"[101344, 104592, 15998]","[nan, nan, nan]","[nan, nan, nan]"
3,104211,2020-09-28 22:44:30.854456+00:00,t,258.831155,79381716002876474,864691136484126764,4,2023-08-17 06:30:26.150664+00:00,t,104211,inhibitory_neuron,STC,"[106224, 103968, 16013]","[nan, nan, nan]","[nan, nan, nan]"
4,104240,2020-09-28 22:41:30.400913+00:00,t,178.378670,79452703222654238,864691135342185905,5,2023-08-17 06:30:26.151422+00:00,t,104240,inhibitory_neuron,ITC,"[106784, 108384, 16286]","[nan, nan, nan]","[nan, nan, nan]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72153,85864,2020-09-28 22:44:51.317074+00:00,t,289.293599,77711420334527629,864691135841447395,72154,2023-08-17 06:40:51.484747+00:00,t,85864,excitatory_neuron,L6CT,"[94080, 242240, 15773]","[nan, nan, nan]","[nan, nan, nan]"
72154,85865,2020-09-28 22:44:51.771785+00:00,t,291.017687,77782957309764253,864691137019541102,72155,2023-08-17 06:40:51.485460+00:00,t,85865,excitatory_neuron,L6CT,"[94336, 251056, 15763]","[nan, nan, nan]","[nan, nan, nan]"
72155,85869,2020-09-28 22:44:45.507917+00:00,t,279.552983,77852570206682268,864691135941178484,72156,2023-08-17 06:40:51.486177+00:00,t,85869,excitatory_neuron,L6b,"[95264, 245264, 16145]","[nan, nan, nan]","[nan, nan, nan]"
72156,85874,2020-09-28 22:41:08.903788+00:00,t,277.481062,77993926170242409,864691136194813910,72157,2023-08-17 06:40:51.486885+00:00,t,85874,excitatory_neuron,L6b,"[96112, 249664, 16080]","[nan, nan, nan]","[nan, nan, nan]"


In [ ]:
def make_tag_entry(taglists, tagdescription_ds, description="tags"):
    unique_tag_lists = [np.unique(tl).tolist() for tl in taglists]
    
    # concatenate all unique tag lists
    unique_tags = np.concatenate(unique_tag_lists).tolist()

    # for each unique_tag_list get the description
    unique_tag_descs = [[tag_d[t] for t in tl] for tl, tag_d in zip(unique_tag_lists, tagdescription_ds)]

    # concatenate all unique tag descriptions
    unique_tag_desc = np.concatenate(unique_tag_descs).tolist()

    # for each tag list, get the indices of the unique tags
    tag_indices = [[unique_tags.index(t) for t in tl] for tl in taglists]
    tag_indices = [list(tags) for tags in zip(*tag_indices)]
    return {"id": "tags",
            "type": "tags",
            "description": description,
            "tags": unique_tags,
            "tag_descriptions": unique_tag_desc,
            "values": tag_indices
           }

unique_ct = ct_df.cell_type.unique()
unique_class = ct_df.classification_system.unique()

ct_desc = {c:c for c in unique_ct}
ct_desc['PTC'] = 'Proximal targetting interneuron'
ct_desc['DTC'] = "Distal targetting interneuron"
ct_desc['ITC'] = "Interneuron targetting interneuron"
ct_desc['STC'] = "Sparsely targetting interneuron"
ct_desc['L5NP']= "Layer 5 Near Projecting"

cell_class_desc = {c:c for c in unique_class}


tag_entry=make_tag_entry([ct_df.cell_type, ct_df.classification_system],
                          [ct_desc, cell_class_desc])

segment_info ={"@type":  "neuroglancer_segment_properties",
        "inline": {
            "ids": [str(i) for i in ct_df.pt_root_id.values.tolist()],
            "properties": [
                {"id": "label",
                 "type": "label",
                 "description": "filename",
                 "values": [str(v) for v in ct_df.pt_root_id.values]},
                tag_entry,
            ]
        }
}
segment_id= client.state.upload_property_json(segment_info)
segment_id
state = client.state

segment_id= client.state.upload_property_json(segment_info)
# this is what you want to add to a neuroglancer segmentation source layer using the plus button
state.build_neuroglancer_url(segment_id,
                            ngl_url = "https://spelunker.cave-explorer.org",
                            format_propeties=True)

: 

In [29]:
client.info.get_datastack_info()

{'aligned_volume': {'id': 1,
  'name': 'minnie65_phase3',
  'display_name': 'Minnie65',
  'description': "This is the second alignment of the IARPA 'minnie65' dataset, completed in the spring of 2020 that used the seamless approach.",
  'image_source': 'precomputed://https://bossdb-open-data.s3.amazonaws.com/iarpa_microns/minnie/minnie65/em'},
 'segmentation_source': 'graphene://https://minnie.microns-daf.com/segmentation/table/minnie3_v1',
 'analysis_database': None,
 'viewer_site': 'https://neuroglancer.neuvue.io/',
 'synapse_table': 'synapses_pni_2',
 'soma_table': 'nucleus_neuron_svm',
 'local_server': 'https://minnie.microns-daf.com',
 'description': 'This is the first version of Minnie that has proofreading enabled. Was first enabled on June 24, 2020. ',
 'viewer_resolution_x': 4.0,
 'viewer_resolution_y': 4.0,
 'viewer_resolution_z': 40.0,
 'proofreading_status_table': None,
 'cell_identification_table': None,
 'proofreading_review_table': None}

In [25]:
client.info.get_datastacks()

['minnie35_public_v0',
 'minnie65_phase3_v0',
 'fanc_sandbox',
 'minnie65_public_v117',
 'minnie65_public_v343',
 'flywire_fafb_sandbox',
 'minnie65_phase2_flat',
 'minnie65_public',
 'minnie35_phase3_v0',
 'basil-seg-aug',
 'minnie65_phase3_v1',
 'minnie65_public_v661',
 'pinky_sandbox',
 'fafb_v15_20221128025036',
 'flywire_fafb_public',
 'minnie65_sandbox']

In [20]:
all_tables = client.annotation.get_tables()
all_tables

['synapses', 'neurons', 'glia', 'nuclei', 'mitochondria']

In [26]:
import numpy as np
import caveclient

# Function to generate dropdown options for datasets
def generate_dataset_dropdown(client):
    available_datasets = client.info.get_datastacks()
    dropdown_options = [{'label': dataset, 'value': dataset} for dataset in available_datasets]
    return dropdown_options

# Function to generate dropdown options for tables
def generate_table_dropdown(client, dataset):
    client = caveclient.CAVEclient(dataset)
    all_tables = client.annotation.get_tables()
    dropdown_options = [{'label': table, 'value': table} for table in all_tables]
    return dropdown_options

# Replace 'pinky_sandbox' with a default dataset to avoid errors before selection
default_dataset = 'pinky_sandbox'
client = caveclient.CAVEclient(default_dataset)

# Generate dropdown options for datasets
dataset_dropdown_options = generate_dataset_dropdown(client)

# Generate dropdown options for tables using the default dataset
default_table_dropdown_options = generate_table_dropdown(client, default_dataset)

# Print the default table dropdown options for verification
print(default_table_dropdown_options)


[{'label': 'synapses', 'value': 'synapses'}, {'label': 'neurons', 'value': 'neurons'}, {'label': 'glia', 'value': 'glia'}, {'label': 'nuclei', 'value': 'nuclei'}, {'label': 'mitochondria', 'value': 'mitochondria'}]
